In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error

%matplotlib inline

plt.style.use('default') # Make the graphs a bit prettier

plt.rcParams['figure.figsize'] = (12, 4)

In [2]:
#pip install -U statsmodels
import statsmodels.api as sm 

/usr/local/lib/python2.7/dist-packages/statsmodels/compat/pandas.py:56: FutureWarning: The pandas.core.datetools module is deprecated and will be removed in a future version. Please use the pandas.tseries module instead.
  from pandas.core import datetools


#  Preprocesamiento de Datos de entrenamiento

El analisis exploratorio que se presenta a continuacion tiene como fin determinar que feathers influyen en mayor medida a la hora de determinar el precio de un inmueble en Capital Federal y Gran Buenos Aires, asi como tambien encontrar datos curiosos en el data set.

Los datos fueron provistos por [Properati](http://www.properati.com.ar) y reorganizo en un solo dataset de la siguiente manera : [GitHub | properati_argentina](https://github.com/BraianVicente/properati_argentina/blob/master/src/TP1%20-%20Properati%20-%20Parseo%20de%20datos%20por%20fecha.ipynb)

In [3]:
sell_properati = pd.read_csv('../data/properati_full_month.csv',low_memory=False)

Completamos la columna 'state_name'

In [4]:
def parse_year_month(date):
    list_date = date.split('-')
    if len(str(list_date[1])) == 1 :
        return str(list_date[0]) + '-0' + str(list_date[1])
    return int(str(list_date[0]) + str(list_date[1]) )

In [5]:
sell_properati['created_on'] = sell_properati.created_on.apply(parse_year_month)

In [6]:
sell_properati['state_name'] = sell_properati.place_with_parent_names.apply(lambda x : x.split('|')[2])

Filtramos unicamente los que nos interesan que corresponde a capital federal y alrededores

In [7]:
sell_properati = sell_properati[
    (sell_properati['state_name'] == 'Bs.As. G.B.A. Zona Norte' ) |
    (sell_properati['state_name'] == 'Bs.As. G.B.A. Zona Sur' ) |
    (sell_properati['state_name'] == 'Bs.As. G.B.A. Zona Oeste' ) |
    (sell_properati['state_name'] == 'Capital Federal' ) 
    ]

In [8]:
sell_properati.shape

(416685, 31)

In [9]:
sell_properati = sell_properati[sell_properati.property_type != 'store']

In [10]:
sell_properati.shape

(408405, 31)

In [11]:
sell_properati['zone'] = sell_properati.place_with_parent_names.apply(lambda x : x.split('|')[3])

Filtramos por los datos que son reelevantes para nuestro analisis, obteniendo unicamente las propiedades de GBA y capital federal

In [12]:
sell_properati.drop(inplace=True,\
    labels=['country_name','description','extra','id', u'image_thumbnail','operation'\
            ,'place_with_parent_names_l1','place_with_parent_names_l2', 'properati_url','surface_in_m2','title',\
           ],axis=1)

Podemos seguir trabajando con los datos que no cuentan con la informacion de Zona utilizando la ubicacion para identificarlos segun el barrio al que pertenecen pero en esta ocacion decidimos dejarlos de lado pues no son una cantidad significativa de datos

#### Las propiedades del tigre son las unicas que no contienen informacion en 'place_name'

In [13]:
sell_properati.loc[pd.isnull(sell_properati['place_name']),'place_name'] = sell_properati.place_name.apply(lambda x : x if pd.notnull(x) else 'Tigre' )

In [14]:
sell_properati = sell_properati[sell_properati['zone'] != '']

In [15]:
#Calcula el precio la incognita (incognita = divisor/dividendo) para todo divisor mayor a cero y todo dividendo mayor a cero.

def dataframe_calc_value(df,incognita,divisor,dividendo):
    df.loc[ (pd.isnull(df[incognita])) |  (df[incognita] == 0 ) & (pd.notnull(df[divisor]) ) & (df[divisor] > 0) & \
            (pd.notnull(df[dividendo]) & (df[dividendo] > 0)) ,incognita] = \
    df.loc[ (pd.isnull(df[incognita])) |  (df[incognita] == 0 ) & (pd.notnull(df[divisor]) ) & (df[divisor] > 0) & \
            (pd.notnull(df[dividendo]) & (df[dividendo] > 0))][divisor] / \
    df.loc[ (pd.isnull(df[incognita])) |  (df[incognita] == 0 ) & (pd.notnull(df[divisor]) ) & (df[divisor] > 0) & \
            (pd.notnull(df[dividendo]) & (df[dividendo] > 0))][dividendo]

## Continuaremos trabajando con el set de datos completo para disponer de toda la informacion.

### Se agrega la superficie total de los inmbuebles que disponen del precio por metro cuadrado en dolares


In [16]:
sell_properati_filter = sell_properati.copy()

In [17]:
dataframe_calc_value(sell_properati_filter,'surface_total_in_m2','price_aprox_usd','price_usd_per_m2')

### Calculamos la superficie total en metros cuadrados de las propiedades que no cuentan con este dato

In [18]:
dataframe_calc_value(sell_properati_filter,'surface_total_in_m2','price','price_per_m2')

### Calculamos el precio por metro cuadrado en USD de los inmuebles que no cuentan con dicha informacion

In [19]:
dataframe_calc_value(sell_properati_filter,'price_usd_per_m2','price_aprox_usd','surface_total_in_m2')

### Se agrega el precio por metro cuadrado en la moneda actual en cada unos de las pripiedades

In [20]:
dataframe_calc_value(sell_properati_filter,'price_per_m2','price','surface_total_in_m2')

In [21]:
sell_properati_filter = sell_properati_filter[~((sell_properati_filter['surface_total_in_m2'] == 0 ) | \
                     (sell_properati_filter['price'] == 0 ))]

Filtraremos los datos que tengan una superficie total en metros cuadrados entre 0 y 1000

In [22]:
filter_properati = sell_properati_filter[(sell_properati_filter['price_aprox_usd'] < 600000) &\
                                         (sell_properati_filter['price_usd_per_m2'] < 5000) & \
                                         (sell_properati_filter['surface_total_in_m2'] < 1000)]

## Le asignamos un numero a cada state_name y cada place_name

df = pd.DataFrame(data=sell_properati.place_name.unique(),columns=['place_name']).reset_index()

df.rename(columns={'index':'id_place_name'},inplace=True)

filter_properati = filter_properati.merge(how='left',left_on='place_name',right_on='place_name',right=df)

df = pd.DataFrame(data=sell_properati.state_name.unique(),columns=['state_name']).reset_index()
df.rename(columns={'index':'id_states'},inplace=True)

filter_properati = filter_properati.merge(how='left',left_on='state_name',right_on='state_name',right=df)

df = pd.DataFrame(data=sell_properati.property_type.unique(),columns=['property_type']).reset_index()
df.rename(columns={'index':'id_property_type'},inplace=True)

filter_properati = filter_properati.merge(how='left',left_on='property_type',right_on='property_type',right=df)

In [23]:
data = filter_properati.drop(axis=1,labels=['currency','lat','lat-lon','lon','place_with_parent_names','price'\
                                            ,'price_aprox_local_currency','price_per_m2','price_usd_per_m2'\
                                            ,'surface_covered_in_m2','zone','geonames_id'])

import re

def devolverInt(x):
        return float(re.search(r'\d+', x).group())

In [24]:
data['expenses'] = data.expenses.apply(lambda x : 1 if pd.notnull(x) else 0 )

In [25]:
data['floor'] = data.floor.apply(lambda x : 1 if pd.notnull(x) else 0 )

In [26]:
data['price_aprox_usd'] = data.price_aprox_usd.apply(lambda x : float(x) if pd.notnull(x) else 0 )

In [27]:
data['rooms'] = data.rooms.apply(lambda x : 1 if pd.notnull(x) else 0 )

In [28]:
data['surface_total_in_m2'] = data.surface_total_in_m2.apply(lambda x : float(x) if pd.notnull(x) else 0 )

# Preprocesamiento de set de test

In [29]:
properati_testing_dataset = pd.read_csv('../data/properati_dataset_testing_noprice.csv',low_memory=False)

In [30]:
properati_testing_dataset['created_on'] = properati_testing_dataset.created_on.apply(parse_year_month)

In [31]:
properati_testing_dataset.loc[- properati_testing_dataset.place_name.isin(sell_properati.place_name.unique()),'place_name'] = \
    properati_testing_dataset[- properati_testing_dataset.place_name.isin(sell_properati.place_name.unique())]['place_with_parent_names']\
    .apply(lambda x : (x.split('|')[3])).copy()

In [32]:
properati_testing_dataset.loc[properati_testing_dataset.property_type == 'departamento','property_type'] = 'apartment'

In [33]:
properati_testing_dataset.loc[properati_testing_dataset.property_type == 'ph','property_type'] = 'PH'

In [34]:
properati_testing_dataset.loc[properati_testing_dataset.property_type == 'casa','property_type'] = 'house'

In [35]:
properati_testing_dataset.drop(inplace=True,\
                              labels=['created_on', 'country_name','description','place_with_parent_names',\
                                      'surface_covered_in_m2','lat-lon','lat','lon'],axis=1)

In [36]:
properati_testing_dataset.loc[(properati_testing_dataset.expenses == 'Sin expensas'),'expenses'] = 0

In [37]:
properati_testing_dataset.loc[(properati_testing_dataset.expenses == 'SI'),'expenses'] = '0'

In [38]:
properati_testing_dataset.loc[(properati_testing_dataset.expenses == 'NO'),'expenses'] = '0'

In [39]:
properati_testing_dataset.loc[pd.isnull(properati_testing_dataset.expenses),'expenses'] = '0 '

In [40]:
properati_testing_dataset.expenses = properati_testing_dataset.expenses.astype(str)

In [41]:
properati_testing_dataset['expenses'] = properati_testing_dataset.expenses.apply(lambda x : 1 if pd.notnull(x) else 0 )

In [42]:
properati_testing_dataset['floor'] = properati_testing_dataset.floor.apply(lambda x : 1 if pd.notnull(x) else 0 )

In [43]:
properati_testing_dataset['rooms'] = properati_testing_dataset.rooms.apply(lambda x : 1 if pd.notnull(x) else 0 )

In [44]:
properati_testing_dataset['surface_total_in_m2'] = properati_testing_dataset.surface_total_in_m2.apply(lambda x : float(x) if pd.notnull(x) else 0 )

In [45]:
ohe_state_name = pd.get_dummies(data['state_name'])
ohe_place_name = pd.get_dummies(data['place_name'])

In [46]:
ohe_property_type = pd.get_dummies(data['property_type'])

In [47]:
data.drop('state_name',axis=1,inplace=True)
data.drop('place_name',axis=1,inplace=True)
data.drop('property_type',axis=1,inplace=True)

In [48]:
data_OHE = data.join(ohe_state_name)

In [49]:
data_OHE = data_OHE.join(ohe_place_name)

In [50]:
data_OHE.shape

(240200, 593)

In [51]:
train_target = data_OHE[['price_aprox_usd']].copy()

In [52]:
test_id = pd.DataFrame(properati_testing_dataset['id'])

In [53]:
ohe_state_name = pd.get_dummies(properati_testing_dataset['state_name'])
ohe_place_name = pd.get_dummies(properati_testing_dataset['place_name'])

In [54]:
ohe_property_type = pd.get_dummies(properati_testing_dataset['property_type'])

In [55]:
properati_testing_dataset.drop('state_name',axis=1,inplace=True)
properati_testing_dataset.drop('place_name',axis=1,inplace=True)
properati_testing_dataset.drop('property_type',axis=1,inplace=True)

In [56]:
properati_testing_dataset_OHE = properati_testing_dataset.join(ohe_state_name)

In [57]:
properati_testing_dataset_OHE = properati_testing_dataset.join(ohe_place_name)

In [58]:
properati_testing_dataset_OHE.drop(inplace=True,axis=1,labels=['id' ,'operation', '', '9 de Abril' ,'El Casco de Leloir','Los Pingüinos Country Club', 'Villa Brown' ,'Villa España', 'Villa Numancia'])

In [59]:
data_OHE = data_OHE[list(properati_testing_dataset_OHE.columns)].copy()

In [60]:
data_OHE.shape

(240200, 216)

In [61]:
properati_testing_dataset_OHE.shape

(14166, 216)

# Normalizacion de datos

In [62]:
train = data_OHE.copy()

In [63]:
testing_data_id = test_id

In [64]:
test = properati_testing_dataset_OHE.copy()

In [65]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 240200 entries, 2 to 597227
Columns: 216 entries, surface_total_in_m2 to William Morris
dtypes: float64(1), int64(3), uint8(212)
memory usage: 57.7 MB


# Reduccion de Dimensiones

from sklearn import decomposition

svd = decomposition.TruncatedSVD(n_components=train.shape[1]-1)

train_transform = svd.fit_transform(train)

pd.DataFrame(svd.explained_variance_).plot()

test_transform = svd.transform(test)

test = pd.DataFrame(test_transform)

train = pd.DataFrame(train_transform)

# Machine Learning


In [66]:
from sklearn import neural_network

In [67]:
MLP = neural_network.MLPRegressor(hidden_layer_sizes=(570,400),max_iter=18)

In [68]:
MLP.fit(train,train_target)

/usr/local/lib/python2.7/dist-packages/sklearn/neural_network/multilayer_perceptron.py:1266: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python2.7/dist-packages/sklearn/neural_network/multilayer_perceptron.py:563: ConvergenceWarning: Stochastic Optimizer: Maximum iterations reached and the optimization hasn't converged yet.
  % (), ConvergenceWarning)


MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(570, 400), learning_rate='constant',
       learning_rate_init=0.001, max_iter=18, momentum=0.9,
       nesterovs_momentum=True, power_t=0.5, random_state=None,
       shuffle=True, solver='adam', tol=0.0001, validation_fraction=0.1,
       verbose=False, warm_start=False)

In [69]:
test_prediction = MLP.predict(test)

In [70]:
predictionDF = pd.DataFrame(test_prediction,columns={"price_usd"})

In [71]:
predictionDF['id'] = testing_data_id['id']

In [72]:
predictionDF.shape

(14166, 2)

In [78]:
predictionDF.sample(5)

,price_usd,id
2587,207934.299000,2279385
2897,110697.658431,2280761
10202,108488.458751,2322762
7449,113979.418669,2303449
6629,126135.714423,2299557


In [76]:
predictionDF.price_usd = predictionDF.price_usd.abs()

In [77]:
predictionDF.to_csv(path_or_buf='../data/prediction_MLP_2',sep=',',header=True,columns=['id','price_usd'],index=False)